In [1]:
import tensorflow as tf
import tensorflow.contrib.learn as learn

dataset = learn.datasets.mnist.read_data_sets('MNIST-data',  one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
image_holder = tf.placeholder(tf.float32, [None, 784])
label_holder= tf.placeholder(tf.float32, [None, 10])


hid_nodes = 200
out_nodes = 10

w0 = tf.Variable(tf.random_normal([784, hid_nodes]))
w1 = tf.Variable(tf.random_normal([hid_nodes, hid_nodes]))
w2 = tf.Variable(tf.random_normal([hid_nodes, hid_nodes]))
w3 = tf.Variable(tf.random_normal([hid_nodes, out_nodes]))

# Define biases
b0 = tf.Variable(tf.random_normal([hid_nodes])) 
b1 = tf.Variable(tf.random_normal([hid_nodes])) 
b2 = tf.Variable(tf.random_normal([hid_nodes]))
b3 = tf.Variable(tf.random_normal([out_nodes]))

layer_1 = tf.add(tf.matmul(image_holder, w0), b0) 
layer_1 = tf.nn.relu(layer_1)
layer_2 = tf.add(tf.matmul(layer_1, w1), b1) 
layer_2 = tf.nn.relu(layer_2) 
layer_3 = tf.add(tf.matmul(layer_2, w2), b2) 
layer_3 = tf.nn.relu(layer_3)
out_layer = tf.matmul(layer_3, w3) + b3
 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out_layer, labels = label_holder))

learning_rate = 0.01
num_epochs = 15
batch_size = 100
num_batches = int(dataset.train.num_examples/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        for batch in range(num_batches):
            
            img_batch, lbl_batch = dataset.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict = {image_holder: img_batch, label_holder: lbl_batch})
#     This checks if the predicted numbers is equal to the label        
    prediction = tf.equal(tf.argmax(out_layer,1), tf.argmax(label_holder, 1))
#     This transforms the numbers to ones and zeros and find the average
    success = tf.reduce_sum(tf.cast(prediction, tf.float32))
    
    print(sess.run(tf.argmax(out_layer,1),feed_dict = {image_holder: img_batch, label_holder: lbl_batch}))
    print(sess.run(success, feed_dict = {image_holder: img_batch, label_holder: lbl_batch}))
 

[6 9 7 1 9 6 2 6 1 1 5 6 4 6 3 9 9 1 1 0 1 2 1 2 9 3 9 2 3 8 0 0 9 9 1 5 6
 9 0 9 3 9 1 9 3 0 6 0 2 1 9 0 2 0 7 2 5 4 1 0 1 8 7 1 8 6 0 4 9 5 4 8 7 2
 6 5 0 4 0 3 3 9 4 3 7 8 7 6 0 5 4 8 0 1 9 0 8 9 6 9]
97.0


In [2]:
from tensorflow.contrib.layers import fully_connected

#setting the image placeholder to a Z x 784; None because we dont know the exact amount of variable it would take
image_holder = tf.placeholder(tf.float32, [None, 784])
label_holder = tf.placeholder(tf.float32, [None, 10])
train = tf.placeholder(tf.bool)

hid_nodes = 200
out_nodes = 10
keep_prob = 0.5

#sets the tensor fully connected to use the normalized function
#sets a boolean value to create layers 
with tf.contrib.framework.arg_scope([tf.contrib.layers.fully_connected],
    normalizer_fn = tf.contrib.layers.batch_norm,
    normalizer_params = {'is_training': train}):
    
    layer1 = fully_connected(image_holder, hid_nodes, scope = 'layer1')
    layer1_drop = tf.layers.dropout(layer1, keep_prob, training = train)
    layer2 = fully_connected(layer1_drop, hid_nodes, scope = 'layer2' )
    layer2_drop = tf.layers.dropout(layer2, keep_prob, training = train)
    layer3 = fully_connected(layer2_drop, hid_nodes, scope = 'layer3' )
    layer3_drop = tf.layers.dropout(layer3, keep_prob, training = train)
    out_layer = fully_connected(layer3_drop, out_nodes, activation_fn = None, scope = 'layer4')
    
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out_layer, labels = label_holder))

learning_rate = 0.01
num_epochs = 15
batch_size = 100
num_batches = int(dataset.train.num_examples/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        for batch in range(num_batches):
            
            img_batch, lbl_batch = dataset.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict = {image_holder: img_batch, label_holder: lbl_batch, train: True})
            
    prediction = tf.equal(tf.argmax(out_layer,1), tf.argmax(label_holder, 1))
    success = tf.reduce_mean(tf.cast(prediction, tf.float32)) 
    
    print(sess.run(tf.argmax(out_layer,0),feed_dict = {image_holder: img_batch, label_holder: lbl_batch, train:False}))
    print(sess.run(prediction, feed_dict = {image_holder: img_batch, label_holder: lbl_batch, train:False}))
    print('Success rate: ', sess.run(success,      
                                     feed_dict={image_holder: dataset.test.images,   
                                                label_holder: dataset.test.labels, train: False}))


Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

[ 4 41 46 11 25  1 86 62 41 69]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True  True  True False  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True False  True  True  True False  True False  True  True
  True  True  True  True  True  True  True False False  True  True  True
  True  True  True  True]


Success rate:  0.9118
